In [114]:
# 필요한 라이브러리를 import

# !pip install flatten-dict
import requests
import json
import pandas as pd
import itertools
import time
import math
import numpy as np
from tqdm import tqdm
from flatten_dict import flatten
from flatten_dict.reducer import make_reducer

In [115]:
# riot-API 이용을 위한 api-key값

api_key = 'RGAPI-6d284b3b-93ef-4397-a516-8530738146fa'

In [116]:
# 각 tier별 game id를 가져오기위해 url을 저장

IRON_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/IV?page='
IRON_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/III?page='
IRON_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/II?page='
IRON_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/I?page='
BRONZE_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/IV?page='
BRONZE_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/III?page='
BRONZE_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/II?page='
BRONZE_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/BRONZE/I?page='
SILVER_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/IV?page='
SILVER_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/III?page='
SILVER_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/II?page='
SILVER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page='
GOLD_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page='
GOLD_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/III?page='
GOLD_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/II?page='
GOLD_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/I?page='
PLATINUM_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/IV?page='
PLATINUM_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/III?page='
PLATINUM_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/II?page='
PLATINUM_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/PLATINUM/I?page='
DIAMOND_URL4='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/IV?page='
DIAMOND_URL3='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/III?page='
DIAMOND_URL2='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/II?page='
DIAMOND_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page='
MASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page='
GRANDMASTER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page='
CHALLENGER_URL1='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page='
MATCH_URL = 'https://kr.api.riotgames.com/lol/match/v4/matches/'

In [117]:
# tier의 urel과 api-key를 입력하여 summonerId를 반환받음 return - array

def summonersT(tier_url, api_key):
    print('----------------------------------------------------Summoners start----------------------------------------------------')
    summoners = []
    for i in range(1,5):
        res = requests.get(tier_url+str(i), headers={"X-Riot-Token": api_key})
        summoners.append(json.loads(res.text)[5:105])
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    print('Waiting for summoners end...')
    time.sleep(120)
    print('----------------------------------------------------Summoners done-----------------------------------------------------')
    return pd.DataFrame(list(itertools.chain(*summoners)))['summonerId'].values


#  summoners(iterable)와 api-key를 입력받아 accountId를 반환 return - array

def getAccountId(summoners, api_key):
    print('\n----------------------------------------------------Accounts start-----------------------------------------------------')
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    accountS = []
    time.sleep(2)
    a = 0
    for i  in tqdm(summoners):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetSummonerId = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/'+str(i)
        res = requests.get(togetSummonerId, headers={"X-Riot-Token": api_key})
        accountS.append(json.loads(res.text))
    print('\n-----------------------------------------------------Accounts done-----------------------------------------------------')   
    return pd.DataFrame(accountS)['accountId'].values


# accountId(iterable)와 api-key를 입력받아 gameId를 반환 return - list

def getMatch(encryptedAccountId, api_key):
    print('\n-----------------------------------------------------MatchId start-----------------------------------------------------')
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = b
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
            print("----Error fixed----")
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    print('\n-----------------------------------------------------MatchId done-----------------------------------------------------')   
    return matchIds


# getMatch중 timeout 오류발생시 다시 request를받기위해 시간간격을 두고 다시 request하도록 함.

def getMatchRe(encryptedAccountId, api_key):
    a = 0
    now = time.gmtime(time.time())
    print('min:',now.tm_min,'sec:',now.tm_sec)
    objects = []
    matchIds = []
    time.sleep(2)
    for i in tqdm(encryptedAccountId):
        a += 1
        if a % 20 == 0:
            time.sleep(1)
        if a % 100 == 0:
            time.sleep(122)
        togetMatchOb = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?endIndex=1&beginIndex=0'.format(str(i))
        res = requests.get(togetMatchOb, headers={"X-Riot-Token": api_key})
        ob = pd.DataFrame(json.loads(res.text))
        if 'matches' in list(ob):
            objects.append(pd.DataFrame(ob)['matches'].values[0])
        else:
            print(ob)
            print("----Error recovery time----")
            time.sleep(90)
            rest=getMatchRe([i], api_key)
            matchIds.append(rest)
    matchIds = matchIds + list(pd.DataFrame(objects)['gameId'].values)
    return matchIds[0]


# tier와 api-key를 입력하면 gameId를 반환하는 함수

def getMatchId(tier, api_key):
    summoners = summonersT(tier, api_key)
    accountId = getAccountId(summoners, api_key)
    gameId = getMatch(accountId, api_key)
    print("Done !!")
    return gameId

### Get Iron games

In [108]:
iron_4_gameId = getMatchId(IRON_URL4, api_key)
iron_3_gameId = getMatchId(IRON_URL3, api_key)
iron_2_gameId = getMatchId(IRON_URL2, api_key)
iron_1_gameId = getMatchId(IRON_URL1, api_key)
np.save('gameId/iron/iron_1_gameId_1', iron_1_gameId)
np.save('gameId/iron/iron_2_gameId_1', iron_2_gameId)
np.save('gameId/iron/iron_3_gameId_1', iron_3_gameId)
np.save('gameId/iron/iron_4_gameId_1', iron_4_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 20 sec: 0
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 22 sec: 0


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:56<00:00,  1.49s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 31 sec: 58


 30%|███████████████████████▌                                                        | 118/400 [02:49<01:52,  2.50it/s]

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 36 sec: 20


...

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 55 sec: 10



 54%|█████████████████████████████████████████▋                                    | 214/400 [07:02<1:27:50, 28.34s/it]

----Error fixed----


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [12:23<00:00,  1.86s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------


### Get Bronze games

In [109]:
bronze_4_gameId = getMatchId(BRONZE_URL4, api_key)
bronze_3_gameId = getMatchId(BRONZE_URL3, api_key)
bronze_2_gameId = getMatchId(BRONZE_URL2, api_key)
bronze_1_gameId = getMatchId(BRONZE_URL1, api_key)
np.save('gameId/bronze/bronze_4_gameId_1', bronze_4_gameId)
np.save('gameId/bronze/bronze_3_gameId_1', bronze_3_gameId)
np.save('gameId/bronze/bronze_2_gameId_1', bronze_2_gameId)
np.save('gameId/bronze/bronze_1_gameId_1', bronze_1_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 0 sec: 34
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 2 sec: 34


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:58<00:00,  1.50s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 12 sec: 35


 36%|████████████████████████████▌                                                   | 143/400 [02:59<01:58,  2.17it/s]

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 17 sec: 6


...


-----------------------------------------------------MatchId done-----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 18 sec: 32
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 20 sec: 32


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:53<00:00,  1.48s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 30 sec: 28


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [10:48<00:00,  1.62s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------


### Get Silver games

In [110]:
silver_4_gameId = getMatchId(SILVER_URL4, api_key)
silver_3_gameId = getMatchId(SILVER_URL3, api_key)
silver_2_gameId = getMatchId(SILVER_URL2, api_key)
silver_1_gameId = getMatchId(SILVER_URL1, api_key)

np.save('gameId/silver/silver_4_gameId_1', silver_4_gameId)
np.save('gameId/silver/silver_3_gameId_1', silver_3_gameId)
np.save('gameId/silver/silver_2_gameId_1', silver_2_gameId)
np.save('gameId/silver/silver_1_gameId_1', silver_1_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 41 sec: 20
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 43 sec: 20


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:53<00:00,  1.48s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 53 sec: 15


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [10:46<00:00,  1.62s/it]



-----------------------------------------------------MatchId done-----------------------------------------------------
----------------------------------------------------Summoners start----------------------------------------------------
min: 4 sec: 5
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 6 sec: 5


...

                          status
message      Rate limit exceeded
status_code                  429
----Error recovery time----
min: 18 sec: 9



 56%|███████████████████████████████████████████▍                                  | 223/400 [08:39<1:23:26, 28.29s/it]

----Error fixed----


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [13:54<00:00,  2.09s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------


### Get Gold games

In [111]:
gold_4_gameId = getMatchId(GOLD_URL4, api_key)
gold_3_gameId = getMatchId(GOLD_URL3, api_key)
gold_2_gameId = getMatchId(GOLD_URL2, api_key)
gold_1_gameId = getMatchId(GOLD_URL1, api_key)

np.save('gameId/gold/gold_4_gameId_1', gold_4_gameId)
np.save('gameId/gold/gold_3_gameId_1', gold_3_gameId)
np.save('gameId/gold/gold_2_gameId_1', gold_2_gameId)
np.save('gameId/gold/gold_1_gameId_1', gold_1_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 23 sec: 27
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 25 sec: 27


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:53<00:00,  1.48s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 35 sec: 23


 31%|█████████████████████████                                                       | 125/400 [02:50<01:50,  2.50it/s]

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 39 sec: 46


...

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 58 sec: 34



 90%|███████████████████████████████████████████████████████████████████████▌        | 358/400 [10:06<19:38, 28.07s/it]

----Error fixed----


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [12:25<00:00,  1.86s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------


### Get platinum games

In [112]:
platinum_4_gameId = getMatchId(PLATINUM_URL4, api_key)
platinum_3_gameId = getMatchId(PLATINUM_URL3, api_key)
platinum_2_gameId = getMatchId(PLATINUM_URL2, api_key)
platinum_1_gameId = getMatchId(PLATINUM_URL1, api_key)

np.save('gameId/platinum/platinum_4_gameId_1', platinum_4_gameId)
np.save('gameId/platinum/platinum_3_gameId_1', platinum_3_gameId)
np.save('gameId/platinum/platinum_2_gameId_1', platinum_2_gameId)
np.save('gameId/platinum/platinum_1_gameId_1', platinum_1_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 0 sec: 58
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 2 sec: 58


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:58<00:00,  1.50s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 12 sec: 58


 86%|████████████████████████████████████████████████████████████████████▍           | 342/400 [08:22<00:27,  2.09it/s]

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 22 sec: 53


...

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 31 sec: 24



 54%|█████████████████████████████████████████▋                                    | 214/400 [07:01<1:27:48, 28.32s/it]

----Error fixed----


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [12:21<00:00,  1.85s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------


### Get diamond, Master, GrandMaster, challenger games

In [118]:
diamond_4_gameId = getMatchId(DIAMOND_URL4, api_key)
diamond_3_gameId = getMatchId(DIAMOND_URL3, api_key)
diamond_2_gameId = getMatchId(DIAMOND_URL2, api_key)
diamond_1_gameId = getMatchId(DIAMOND_URL1, api_key)
master_gameId = getMatchId(MASTER_URL1, api_key)
grandmaster_gameId = getMatchId(GRANDMASTER_URL1, api_key)
challenger_gameId = getMatchId(CHALLENGER_URL1, api_key)

np.save('gameId/diamond/diamond_4_gameId_1', diamond_4_gameId)
np.save('gameId/diamond/diamond_3_gameId_1', diamond_3_gameId)
np.save('gameId/diamond/diamond_2_gameId_1', diamond_2_gameId)
np.save('gameId/diamond/diamond_1_gameId_1', diamond_1_gameId)
np.save('gameId/master/master_gameId_1', master_gameId)
np.save('gameId/grandmaster/grandmaster_gameId_1', grandmaster_gameId)
np.save('gameId/challenger/challenger_gameId_1', challenger_gameId)

----------------------------------------------------Summoners start----------------------------------------------------
min: 23 sec: 34
Waiting for summoners end...
----------------------------------------------------Summoners done-----------------------------------------------------

----------------------------------------------------Accounts start-----------------------------------------------------
min: 25 sec: 34


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [09:54<00:00,  1.49s/it]



-----------------------------------------------------Accounts done-----------------------------------------------------

-----------------------------------------------------MatchId start-----------------------------------------------------
min: 35 sec: 31


 48%|██████████████████████████████████████                                          | 190/400 [03:17<01:15,  2.79it/s]

                      status
message      Gateway timeout
status_code              504
----Error recovery time----
min: 40 sec: 21


...

                          status
message      Rate limit exceeded
status_code                  429
----Error recovery time----
min: 42 sec: 58



 49%|███████████████████████████████████████▊                                          | 33/68 [01:44<16:20, 28.00s/it]

----Error fixed----


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [01:58<00:00,  1.75s/it]


-----------------------------------------------------MatchId done-----------------------------------------------------
Done !!


# Testing

In [123]:
a = np.load('gameId/challenger/challenger_gameId_1.npy')
len(a)

68

In [124]:
matches=[]
for i in tqdm(a):
    load_url = MATCH_URL + str(i)
    res = requests.get(load_url, headers={"X-Riot-Token": api_key})
    matches.append(json.loads(res.text))

100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:25<00:00,  2.68it/s]


In [125]:
def gameidLoader(path):
    gameIds = np.load(path)
    
    
    
def flatter(matches):
    matchlist = []
    for i in matches:
        matchlist.append(flatten(i, enumerate_types=(list,), reducer=make_reducer(delimiter='_')))
    
    return matchlist


def filter_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            newDict[key] = value
    return newDict



def filter_team_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    ind = -1
    kl = ['teams_win', 'teams_firstBlood', 'teams_firstTower', 'teams_firstInhibitor', 'teams_firstBaron', 'teams_firstDragon', 'teams_firstRiftHerald', 'teams_towerKills', 'teams_inhibitorKills', 'teams_baronKills', 'teams_dragonKills', 'teams_riftHeraldKills']
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            ind += 1
            newDict[kl[ind]] = value
    return newDict



def filter_participant_dict(d, f):
    ''' Filters dictionary d by function f. '''
    newDict = dict()
    # Iterate over all (k,v) pairs in names
    for key, value in d.items():
        # Is condition satisfied?
        if f(key, value):
            pkey = 'avg_'+key[14:]
            newDict[pkey] = value
    return newDict

def winTeams_0(match):
    matchtemp = match
    
    for delstring in ['item', 'Id', 'playerScore', 'perk', 'Perk', 'bans', 'PlayerScore', 'timeline_csDiffPerMinDeltas_0-10', 'stats_totalScoreRank', 'dominionVictoryScore', 'stats_win', 'stats_unrealKills', 'stats_sightWardsBoughtInGame', 'timeline', 'vilemawKills', 'dominionVictoryScore','stats_firstInhibitorAssist','stats_firstInhibitorKill', 'stats_firstTowerAssist', 'stats_firstTowerKill', 'stats_firstBloodAssist', 'stats_firstBloodKill']: 
        temp = list(filter_dict(match, lambda k,v: delstring in k).keys())
        for i in temp:
            del(matchtemp[i])
    
    
    avg_data = (np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_0' in k).values())) 
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_1' in k).values())) 
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_2' in k).values())) 
    +np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_3' in k).values()))
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_4' in k).values())))/5
    avg_label = list(filter_participant_dict(matchtemp, lambda k,v: 'participants_0' in k).keys())
    win = {avg_label[i]: avg_data[i] for i in range(len(avg_label))}
        
        
    avg_data = (np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_5' in k).values())) 
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_6' in k).values())) 
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_7' in k).values())) 
    +np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_8' in k).values()))
    + np.array(list(filter_participant_dict(matchtemp, lambda k,v: 'participants_9' in k).values())))/5
    loss = {avg_label[i]: avg_data[i] for i in range(len(avg_label))}
    
    win.update(filter_team_dict(matchtemp, lambda k,v: 'teams_0' in k))
    loss.update(filter_team_dict(matchtemp, lambda k,v: 'teams_1' in k))
    
    return [win, loss]


def winTeams_1(match):
    temp = winTeams_0(match)
    return [temp[1], temp[0]]


def divideTeams(matches):
    resultList = list()
    for i in matches:
        lossDict = dict()
        winDict = dict()
        lossDict.update({'gameDuration':i['gameDuration']})
        winDict.update({'gameDuration':i['gameDuration']})
        lossDict.update({'gameMode':i['gameMode']})
        winDict.update({'gameMode':i['gameMode']})
        
        if i['teams_0_win'] == 'Win':
            winDict.update(winTeams_0(i)[0])
            lossDict.update(winTeams_0(i)[1])
        else:
            winDict.update(winTeams_1(i)[0])
            lossDict.update(winTeams_1(i)[1])
        resultList.append(winDict)
        resultList.append(lossDict)
    return resultList   

def preprocessing(matches):
    df = pd.DataFrame(divideTeams(flatter(matches)))
    df["teams_firstBlood"] = df["teams_firstBlood"].astype(int)
    df["teams_firstTower"] = df["teams_firstTower"].astype(int)
    df["teams_firstInhibitor"] = df["teams_firstInhibitor"].astype(int)
    df["teams_firstBaron"] = df["teams_firstBaron"].astype(int)
    df["teams_firstDragon"] = df["teams_firstDragon"].astype(int)
    df["teams_firstRiftHerald"] = df["teams_firstRiftHerald"].astype(int)
    df = df.loc[df['gameMode'] == 'CLASSIC']
    df.insert(0, 'team_win', df.teams_win)
    df = df.drop(columns=['teams_win', 'gameMode'])
    return df



IndentationError: expected an indented block (<ipython-input-125-d52bb186dca9>, line 4)

In [103]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None


df = preprocessing(matches)
df


,team_win,gameDuration,avg__stats_kills,avg__stats_deaths,avg__stats_assists,avg__stats_largestKillingSpree,avg__stats_largestMultiKill,avg__stats_killingSprees,avg__stats_longestTimeSpentLiving,avg__stats_doubleKills,avg__stats_tripleKills,avg__stats_quadraKills,avg__stats_pentaKills,avg__stats_totalDamageDealt,avg__stats_magicDamageDealt,avg__stats_physicalDamageDealt,avg__stats_trueDamageDealt,avg__stats_largestCriticalStrike,avg__stats_totalDamageDealtToChampions,avg__stats_magicDamageDealtToChampions,avg__stats_physicalDamageDealtToChampions,avg__stats_trueDamageDealtToChampions,avg__stats_totalHeal,avg__stats_totalUnitsHealed,avg__stats_damageSelfMitigated,avg__stats_damageDealtToObjectives,avg__stats_damageDealtToTurrets,avg__stats_visionScore,avg__stats_timeCCingOthers,avg__stats_totalDamageTaken,avg__stats_magicalDamageTaken,avg__stats_physicalDamageTaken,avg__stats_trueDamageTaken,avg__stats_goldEarned,avg__stats_goldSpent,avg__stats_turretKills,avg__stats_inhibitorKills,avg__stats_totalMinionsKilled,avg__stats_neutralMinionsKilled,avg__stats_totalTimeCrowdControlDealt,avg__stats_champLevel,avg__stats_visionWardsBoughtInGame,teams_firstBlood,teams_firstTower,teams_firstInhibitor,teams_firstBaron,teams_firstDragon,teams_firstRiftHerald,teams_towerKills,teams_inhibitorKills,teams_baronKills,teams_dragonKills,teams_riftHeraldKills,avg__stats_neutralMinionsKilledTeamJungle,avg__stats_neutralMinionsKilledEnemyJungle,avg__stats_wardsPlaced,avg__stats_wardsKilled
2,Win,1877,6.8,4.8,12.4,3.8,1.6,1.4,665.6,0.8,0.2,0.0,0.0,130861.0,11894.6,113624.4,5341.2,271.4,14845.6,2332.0,11935.0,577.8,5006.4,2.4,14695.8,11145.2,1815.6,46.4,21.6,19172.4,6875.4,11320.6,975.4,12183.4,11455.0,0.6,0.0,139.6,41.0,258.4,15.8,6.4,1,0,0,0,0,0,3,0,0,3,1,28.0,1.6,17.2,12.0
3,Fail,1877,4.8,6.8,5.2,3.2,1.8,1.2,685.6,0.8,0.2,0.0,0.0,142871.2,35593.0,99245.2,8032.0,455.8,14122.4,6504.0,6644.6,973.4,5188.8,1.4,15556.6,13240.6,1669.0,45.4,14.8,20404.6,3140.8,16685.0,577.8,12574.0,12220.0,0.8,0.0,152.2,46.4,122.8,14.6,6.0,0,1,0,1,1,1,4,0,1,2,1,28.8,4.8,22.2,8.8
4,Win,966,3.8,0.8,5.0,2.8,1.6,1.0,220.6,0.8,0.0,0.0,0.0,58019.4,24249.4,27265.4,6503.8,77.6,7637.8,2879.8,4112.8,644.0,2997.8,1.0,4598.2,3875.6,1324.6,14.2,7.0,8508.2,1205.0,6625.6,676.4,6749.6,5045.0,0.4,0.0,87.8,19.6,76.8,10.6,2.2,1,1,0,0,1,0,2,0,0,2,0,14.4,0.2,7.0,1.6
5,Fail,966,0.8,3.8,0.6,0.4,0.6,0.2,385.4,0.0,0.0,0.0,0.0,41220.0,7829.2,26561.4,6828.6,45.0,5019.6,951.4,3391.2,676.6,2300.6,1.8,5557.6,2003.8,155.4,8.4,6.8,10509.0,3166.2,6698.4,643.8,4657.4,4290.0,0.0,0.0,66.6,18.8,112.2,9.2,1.0,0,0,0,0,0,1,0,0,0,0,1,14.6,0.2,4.4,1.4
6,Win,953,5.2,2.0,6.2,2.6,1.4,1.6,412.6,0.6,0.2,0.0,0.0,60611.6,19727.4,35421.0,5462.4,168.6,7933.6,2895.0,4106.0,931.6,4264.2,2.4,6059.8,6393.6,2165.6,14.4,11.2,10170.6,2957.6,7026.4,186.2,7344.2,5875.0,0.6,0.0,73.6,27.6,98.2,10.2,1.8,1,1,0,0,1,1,3,0,0,2,1,14.0,6.4,7.2,1.2
7,Fail,953,2.0,5.2,2.8,0.4,0.8,0.4,309.4,0.0,0.0,0.0,0.0,37337.8,15997.6,19461.4,1878.4,111.4,5902.8,2681.4,3034.4,186.2,1594.6,1.8,5552.2,1411.4,422.4,10.2,10.6,9679.8,3015.2,5732.4,931.4,5328.8,4550.0,0.0,0.0,64.2,12.4,141.2,8.8,1.2,0,0,0,0,0,0,0,0,0,0,0,10.8,0.0,4.8,2.0
8,Win,2319,9.4,8.8,19.6,2.4,2.2,2.4,489.4,1.4,0.2,0.2,0.0,185698.0,46706.8,126121.6,12868.4,327.8,30360.4,12910.0,13795.4,3654.4,10074.6,3.8,27305.8,17357.8,3663.0,30.8,35.6,34674.8,15807.6,16709.8,2156.4,16822.8,17128.0,1.6,0.2,154.8,47.6,217.4,17.8,4.2,1,1,0,0,1,1,8,1,2,5,1,30.2,5.2,13.8,5.6
9,Fail,2319,8.8,9.4,17.6,3.4,2.0,1.6,550.2,1.6,0.4,0.2,0.0,178415.6,82562.8,85724.0,10127.6,185.6,27369.2,15111.8,10100.0,2156.2,8746.4,1.0,28819.0,16850.4,3825.4,42.6,26.0,36324.0,13648.8,19019.8,3654.0,15868.6,15922.0,1.4,0.2,144.2,46.2,499.2,17.2,5.6,0,0,1,1,0,0,9,1,1,1,1,25.8,8.4,17.0,4.4
10,Win,1927,5.4,4.4,8.6,3.6,1.2,1.4,831.0,0.4,0.0,0.0,0.0,154726.6,38780.6,105469.8,10475.0,289.6,16062.6,5597.4,8645.2,1819.2,3867.6,2.0,21650.6,13208.8,3499.2,38.8,27.8,20152.0,5182.6,12895.8,2072.6,13361.0,11630.0,1.8,0.4,158.6,42.8,466.0,16.0,6.0,1

In [77]:
gameData = []
for i in matches:
    gameData.append(flatten(i, enumerate_types=(list,), reducer=make_reducer(delimiter='_')))

In [354]:
a = {'asd' : 10, 'sdf' : 20}
b = {'asd' : 100, 'sdf' : 220}
c = np.array(list(b.values())) + np.array(list(a.values()))
c

array([110, 240])

In [128]:

tes = pd.DataFrame(gameData)
aaaaaa = tes.keys()
for i in aaaaaa:
    print(i)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
def divTeam(gamedata):
gameDuration
teams_0_teamId
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_riftHeraldKills



participants_0_participantId
participants_0_teamId
participants_0_championId
participants_0_spell1Id
participants_0_spell2Id
participants_0_stats_participantId
participants_0_stats_win
participants_0_stats_item0
participants_0_stats_item1
participants_0_stats_item2
participants_0_stats_item3
participants_0_stats_item4
participants_0_stats_item5
participants_0_stats_item6
participants_0_stats_kills
participants_0_stats_deaths
participants_0_stats_assists
participants_0_stats_largestKillingSpree
participants_0_stats_largestMultiKill
participants_0_stats_killingSprees
participants_0_stats_longestTimeSpentLiving
participants_0_stats_doubleKills
participants_0_stats_tripleKills
participants_0_stats_quadraKills
participants_0_stats_pentaKills
participants_0_stats_unrealKills
participants_0_stats_totalDamageDealt
participants_0_stats_magicDamageDealt
participants_0_stats_physicalDamageDealt
participants_0_stats_trueDamageDealt
participants_0_stats_largestCriticalStrike
participants_0_stats_totalDamageDealtToChampions
participants_0_stats_magicDamageDealtToChampions
participants_0_stats_physicalDamageDealtToChampions
participants_0_stats_trueDamageDealtToChampions
participants_0_stats_totalHeal
participants_0_stats_totalUnitsHealed
participants_0_stats_damageSelfMitigated
participants_0_stats_damageDealtToObjectives
participants_0_stats_damageDealtToTurrets
participants_0_stats_visionScore
participants_0_stats_timeCCingOthers
participants_0_stats_totalDamageTaken
participants_0_stats_magicalDamageTaken
participants_0_stats_physicalDamageTaken
participants_0_stats_trueDamageTaken
participants_0_stats_goldEarned
participants_0_stats_goldSpent
participants_0_stats_turretKills
participants_0_stats_inhibitorKills
participants_0_stats_totalMinionsKilled
participants_0_stats_neutralMinionsKilled
participants_0_stats_totalTimeCrowdControlDealt
participants_0_stats_champLevel
participants_0_stats_visionWardsBoughtInGame
participants_0_stats_sightWardsBoughtInGame
participants_0_stats_firstBloodKill
participants_0_stats_firstBloodAssist
participants_0_stats_firstTowerKill
participants_0_stats_firstTowerAssist
participants_0_stats_firstInhibitorKill
participants_0_stats_firstInhibitorAssist
participants_0_stats_combatPlayerScore
participants_0_stats_objectivePlayerScore
participants_0_stats_totalPlayerScore
participants_0_stats_totalScoreRank
participants_0_stats_playerScore0
participants_0_stats_playerScore1
participants_0_stats_playerScore2
participants_0_stats_playerScore3
participants_0_stats_playerScore4
participants_0_stats_playerScore5
participants_0_stats_playerScore6
participants_0_stats_playerScore7
participants_0_stats_playerScore8
participants_0_stats_playerScore9
participants_0_stats_perk0
participants_0_stats_perk0Var1
participants_0_stats_perk0Var2
participants_0_stats_perk0Var3
participants_0_stats_perk1
participants_0_stats_perk1Var1
participants_0_stats_perk1Var2
participants_0_stats_perk1Var3
participants_0_stats_perk2
participants_0_stats_perk2Var1
participants_0_stats_perk2Var2
participants_0_stats_perk2Var3
participants_0_stats_perk3
participants_0_stats_perk3Var1
participants_0_stats_perk3Var2
participants_0_stats_perk3Var3
participants_0_stats_perk4
participants_0_stats_perk4Var1
participants_0_stats_perk4Var2
participants_0_stats_perk4Var3
participants_0_stats_perk5
participants_0_stats_perk5Var1
participants_0_stats_perk5Var2
participants_0_stats_perk5Var3
participants_0_stats_perkPrimaryStyle
participants_0_stats_perkSubStyle
participants_0_stats_statPerk0
participants_0_stats_statPerk1
participants_0_stats_statPerk2
participants_0_timeline_participantId
participants_0_timeline_creepsPerMinDeltas_0-10
participants_0_timeline_xpPerMinDeltas_0-10
participants_0_timeline_goldPerMinDeltas_0-10
participants_0_timeline_csDiffPerMinDeltas_0-10
participants_0_timeline_xpDiffPerMinDeltas_0-10
participants_0_timeline_damageTakenPerMinDeltas_0-10
participants_0_timeline_damageTakenDiffPerMinDeltas_0-10
participants_0_timeline_role
participants_0_timeline_lane
participants_1_participantId
participants_1_teamId
participants_1_championId
participants_1_spell1Id
participants_1_spell2Id
participants_1_stats_participantId
participants_1_stats_win
participants_1_stats_item0
participants_1_stats_item1
participants_1_stats_item2
participants_1_stats_item3
participants_1_stats_item4
participants_1_stats_item5
participants_1_stats_item6
participants_1_stats_kills
participants_1_stats_deaths
participants_1_stats_assists
participants_1_stats_largestKillingSpree
participants_1_stats_largestMultiKill
participants_1_stats_killingSprees
participants_1_stats_longestTimeSpentLiving
participants_1_stats_doubleKills
participants_1_stats_tripleKills
participants_1_stats_quadraKills
participants_1_stats_pentaKills
participants_1_stats_unrealKills
participants_1_stats_totalDamageDealt
participants_1_stats_magicDamageDealt
participants_1_stats_physicalDamageDealt
participants_1_stats_trueDamageDealt
participants_1_stats_largestCriticalStrike
participants_1_stats_totalDamageDealtToChampions
participants_1_stats_magicDamageDealtToChampions
participants_1_stats_physicalDamageDealtToChampions
participants_1_stats_trueDamageDealtToChampions
participants_1_stats_totalHeal
participants_1_stats_totalUnitsHealed
participants_1_stats_damageSelfMitigated
participants_1_stats_damageDealtToObjectives
participants_1_stats_damageDealtToTurrets
participants_1_stats_visionScore
participants_1_stats_timeCCingOthers
participants_1_stats_totalDamageTaken
participants_1_stats_magicalDamageTaken
participants_1_stats_physicalDamageTaken
participants_1_stats_trueDamageTaken
participants_1_stats_goldEarned
participants_1_stats_goldSpent
participants_1_stats_turretKills
participants_1_stats_inhibitorKills
participants_1_stats_totalMinionsKilled
participants_1_stats_neutralMinionsKilled
participants_1_stats_totalTimeCrowdControlDealt
participants_1_stats_champLevel
participants_1_stats_visionWardsBoughtInGame
participants_1_stats_sightWardsBoughtInGame
participants_1_stats_firstBloodKill
participants_1_stats_firstBloodAssist
participants_1_stats_firstTowerKill
participants_1_stats_firstTowerAssist
participants_1_stats_firstInhibitorKill
participants_1_stats_firstInhibitorAssist
participants_1_stats_combatPlayerScore
participants_1_stats_objectivePlayerScore
participants_1_stats_totalPlayerScore
participants_1_stats_totalScoreRank
participants_1_stats_playerScore0
participants_1_stats_playerScore1
participants_1_stats_playerScore2
participants_1_stats_playerScore3
participants_1_stats_playerScore4
participants_1_stats_playerScore5
participants_1_stats_playerScore6
participants_1_stats_playerScore7
participants_1_stats_playerScore8
participants_1_stats_playerScore9
participants_1_stats_perk0
participants_1_stats_perk0Var1
participants_1_stats_perk0Var2
participants_1_stats_perk0Var3
participants_1_stats_perk1
participants_1_stats_perk1Var1
participants_1_stats_perk1Var2
participants_1_stats_perk1Var3
participants_1_stats_perk2
participants_1_stats_perk2Var1
participants_1_stats_perk2Var2
participants_1_stats_perk2Var3
participants_1_stats_perk3
participants_1_stats_perk3Var1
participants_1_stats_perk3Var2
participants_1_stats_perk3Var3
participants_1_stats_perk4
participants_1_stats_perk4Var1
participants_1_stats_perk4Var2
participants_1_stats_perk4Var3
participants_1_stats_perk5
participants_1_stats_perk5Var1
participants_1_stats_perk5Var2
participants_1_stats_perk5Var3
participants_1_stats_perkPrimaryStyle
participants_1_stats_perkSubStyle
participants_1_stats_statPerk0
participants_1_stats_statPerk1
participants_1_stats_statPerk2
participants_1_timeline_participantId
participants_1_timeline_creepsPerMinDeltas_0-10
participants_1_timeline_xpPerMinDeltas_0-10
participants_1_timeline_goldPerMinDeltas_0-10
participants_1_timeline_csDiffPerMinDeltas_0-10
participants_1_timeline_xpDiffPerMinDeltas_0-10
participants_1_timeline_damageTakenPerMinDeltas_0-10
participants_1_timeline_damageTakenDiffPerMinDeltas_0-10
participants_1_timeline_role
participants_1_timeline_lane
participants_2_participantId
participants_2_teamId
participants_2_championId
participants_2_spell1Id
participants_2_spell2Id
participants_2_stats_participantId
participants_2_stats_win
participants_2_stats_item0
participants_2_stats_item1
participants_2_stats_item2
participants_2_stats_item3
participants_2_stats_item4
participants_2_stats_item5
participants_2_stats_item6
participants_2_stats_kills
participants_2_stats_deaths
participants_2_stats_assists
participants_2_stats_largestKillingSpree
participants_2_stats_largestMultiKill
participants_2_stats_killingSprees
participants_2_stats_longestTimeSpentLiving
participants_2_stats_doubleKills
participants_2_stats_tripleKills
participants_2_stats_quadraKills
participants_2_stats_pentaKills
participants_2_stats_unrealKills
participants_2_stats_totalDamageDealt
participants_2_stats_magicDamageDealt
participants_2_stats_physicalDamageDealt
participants_2_stats_trueDamageDealt
participants_2_stats_largestCriticalStrike
participants_2_stats_totalDamageDealtToChampions
participants_2_stats_magicDamageDealtToChampions
participants_2_stats_physicalDamageDealtToChampions
participants_2_stats_trueDamageDealtToChampions
participants_2_stats_totalHeal
participants_2_stats_totalUnitsHealed
participants_2_stats_damageSelfMitigated
participants_2_stats_damageDealtToObjectives
participants_2_stats_damageDealtToTurrets
participants_2_stats_visionScore
participants_2_stats_timeCCingOthers
participants_2_stats_totalDamageTaken
participants_2_stats_magicalDamageTaken
participants_2_stats_physicalDamageTaken
participants_2_stats_trueDamageTaken
participants_2_stats_goldEarned
participants_2_stats_goldSpent
participants_2_stats_turretKills
participants_2_stats_inhibitorKills
participants_2_stats_totalMinionsKilled
participants_2_stats_neutralMinionsKilled
participants_2_stats_totalTimeCrowdControlDealt
participants_2_stats_champLevel
participants_2_stats_visionWardsBoughtInGame
participants_2_stats_sightWardsBoughtInGame
participants_2_stats_firstBloodKill
participants_2_stats_firstBloodAssist
participants_2_stats_firstTowerKill
participants_2_stats_firstTowerAssist
participants_2_stats_firstInhibitorKill
participants_2_stats_firstInhibitorAssist
participants_2_stats_combatPlayerScore
participants_2_stats_objectivePlayerScore
participants_2_stats_totalPlayerScore
participants_2_stats_totalScoreRank
participants_2_stats_playerScore0
participants_2_stats_playerScore1
participants_2_stats_playerScore2
participants_2_stats_playerScore3
participants_2_stats_playerScore4
participants_2_stats_playerScore5
participants_2_stats_playerScore6
participants_2_stats_playerScore7
participants_2_stats_playerScore8
participants_2_stats_playerScore9
participants_2_stats_perk0
participants_2_stats_perk0Var1
participants_2_stats_perk0Var2
participants_2_stats_perk0Var3
participants_2_stats_perk1
participants_2_stats_perk1Var1
participants_2_stats_perk1Var2
participants_2_stats_perk1Var3
participants_2_stats_perk2
participants_2_stats_perk2Var1
participants_2_stats_perk2Var2
participants_2_stats_perk2Var3
participants_2_stats_perk3
participants_2_stats_perk3Var1
participants_2_stats_perk3Var2
participants_2_stats_perk3Var3
participants_2_stats_perk4
participants_2_stats_perk4Var1
participants_2_stats_perk4Var2
participants_2_stats_perk4Var3
participants_2_stats_perk5
participants_2_stats_perk5Var1
participants_2_stats_perk5Var2
participants_2_stats_perk5Var3
participants_2_stats_perkPrimaryStyle
participants_2_stats_perkSubStyle
participants_2_stats_statPerk0
participants_2_stats_statPerk1
participants_2_stats_statPerk2
participants_2_timeline_participantId
participants_2_timeline_creepsPerMinDeltas_0-10
participants_2_timeline_xpPerMinDeltas_0-10
participants_2_timeline_goldPerMinDeltas_0-10
participants_2_timeline_csDiffPerMinDeltas_0-10
participants_2_timeline_xpDiffPerMinDeltas_0-10
participants_2_timeline_damageTakenPerMinDeltas_0-10
participants_2_timeline_damageTakenDiffPerMinDeltas_0-10
participants_2_timeline_role
participants_2_timeline_lane
participants_3_participantId
participants_3_teamId
participants_3_championId
participants_3_spell1Id
participants_3_spell2Id
participants_3_stats_participantId
participants_3_stats_win
participants_3_stats_item0
participants_3_stats_item1
participants_3_stats_item2
participants_3_stats_item3
participants_3_stats_item4
participants_3_stats_item5
participants_3_stats_item6
participants_3_stats_kills
participants_3_stats_deaths
participants_3_stats_assists
participants_3_stats_largestKillingSpree
participants_3_stats_largestMultiKill
participants_3_stats_killingSprees
participants_3_stats_longestTimeSpentLiving
participants_3_stats_doubleKills
participants_3_stats_tripleKills
participants_3_stats_quadraKills
participants_3_stats_pentaKills
participants_3_stats_unrealKills
participants_3_stats_totalDamageDealt
participants_3_stats_magicDamageDealt
participants_3_stats_physicalDamageDealt
participants_3_stats_trueDamageDealt
participants_3_stats_largestCriticalStrike
participants_3_stats_totalDamageDealtToChampions
participants_3_stats_magicDamageDealtToChampions
participants_3_stats_physicalDamageDealtToChampions
participants_3_stats_trueDamageDealtToChampions
participants_3_stats_totalHeal
participants_3_stats_totalUnitsHealed
participants_3_stats_damageSelfMitigated
participants_3_stats_damageDealtToObjectives
participants_3_stats_damageDealtToTurrets
participants_3_stats_visionScore
participants_3_stats_timeCCingOthers
participants_3_stats_totalDamageTaken
participants_3_stats_magicalDamageTaken
participants_3_stats_physicalDamageTaken
participants_3_stats_trueDamageTaken
participants_3_stats_goldEarned
participants_3_stats_goldSpent
participants_3_stats_turretKills
participants_3_stats_inhibitorKills
participants_3_stats_totalMinionsKilled
participants_3_stats_neutralMinionsKilled
participants_3_stats_totalTimeCrowdControlDealt
participants_3_stats_champLevel
participants_3_stats_visionWardsBoughtInGame
participants_3_stats_sightWardsBoughtInGame
participants_3_stats_firstBloodKill
participants_3_stats_firstBloodAssist
participants_3_stats_firstTowerKill
participants_3_stats_firstTowerAssist
participants_3_stats_firstInhibitorKill
participants_3_stats_firstInhibitorAssist
participants_3_stats_combatPlayerScore
participants_3_stats_objectivePlayerScore
participants_3_stats_totalPlayerScore
participants_3_stats_totalScoreRank
participants_3_stats_playerScore0
participants_3_stats_playerScore1
participants_3_stats_playerScore2
participants_3_stats_playerScore3
participants_3_stats_playerScore4
participants_3_stats_playerScore5
participants_3_stats_playerScore6
participants_3_stats_playerScore7
participants_3_stats_playerScore8
participants_3_stats_playerScore9
participants_3_stats_perk0
participants_3_stats_perk0Var1
participants_3_stats_perk0Var2
participants_3_stats_perk0Var3
participants_3_stats_perk1
participants_3_stats_perk1Var1
participants_3_stats_perk1Var2
participants_3_stats_perk1Var3
participants_3_stats_perk2
participants_3_stats_perk2Var1
participants_3_stats_perk2Var2
participants_3_stats_perk2Var3
participants_3_stats_perk3
participants_3_stats_perk3Var1
participants_3_stats_perk3Var2
participants_3_stats_perk3Var3
participants_3_stats_perk4
participants_3_stats_perk4Var1
participants_3_stats_perk4Var2
participants_3_stats_perk4Var3
participants_3_stats_perk5
participants_3_stats_perk5Var1
participants_3_stats_perk5Var2
participants_3_stats_perk5Var3
participants_3_stats_perkPrimaryStyle
participants_3_stats_perkSubStyle
participants_3_stats_statPerk0
participants_3_stats_statPerk1
participants_3_stats_statPerk2
participants_3_timeline_participantId
participants_3_timeline_creepsPerMinDeltas_0-10
participants_3_timeline_xpPerMinDeltas_0-10
participants_3_timeline_goldPerMinDeltas_0-10
participants_3_timeline_csDiffPerMinDeltas_0-10
participants_3_timeline_xpDiffPerMinDeltas_0-10
participants_3_timeline_damageTakenPerMinDeltas_0-10
participants_3_timeline_damageTakenDiffPerMinDeltas_0-10
participants_3_timeline_role
participants_3_timeline_lane
participants_4_participantId
participants_4_teamId
participants_4_championId
participants_4_spell1Id
participants_4_spell2Id
participants_4_stats_participantId
participants_4_stats_win
participants_4_stats_item0
participants_4_stats_item1
participants_4_stats_item2
participants_4_stats_item3
participants_4_stats_item4
participants_4_stats_item5
participants_4_stats_item6
participants_4_stats_kills
participants_4_stats_deaths
participants_4_stats_assists
participants_4_stats_largestKillingSpree
participants_4_stats_largestMultiKill
participants_4_stats_killingSprees
participants_4_stats_longestTimeSpentLiving
participants_4_stats_doubleKills
participants_4_stats_tripleKills
participants_4_stats_quadraKills
participants_4_stats_pentaKills
participants_4_stats_unrealKills
participants_4_stats_totalDamageDealt
participants_4_stats_magicDamageDealt
participants_4_stats_physicalDamageDealt
participants_4_stats_trueDamageDealt
participants_4_stats_largestCriticalStrike
participants_4_stats_totalDamageDealtToChampions
participants_4_stats_magicDamageDealtToChampions
participants_4_stats_physicalDamageDealtToChampions
participants_4_stats_trueDamageDealtToChampions
participants_4_stats_totalHeal
participants_4_stats_totalUnitsHealed
participants_4_stats_damageSelfMitigated
participants_4_stats_damageDealtToObjectives
participants_4_stats_damageDealtToTurrets
participants_4_stats_visionScore
participants_4_stats_timeCCingOthers
participants_4_stats_totalDamageTaken
participants_4_stats_magicalDamageTaken
participants_4_stats_physicalDamageTaken
participants_4_stats_trueDamageTaken
participants_4_stats_goldEarned
participants_4_stats_goldSpent
participants_4_stats_turretKills
participants_4_stats_inhibitorKills
participants_4_stats_totalMinionsKilled
participants_4_stats_neutralMinionsKilled
participants_4_stats_totalTimeCrowdControlDealt
participants_4_stats_champLevel
participants_4_stats_visionWardsBoughtInGame
participants_4_stats_sightWardsBoughtInGame
participants_4_stats_firstBloodKill
participants_4_stats_firstBloodAssist
participants_4_stats_firstTowerKill
participants_4_stats_firstTowerAssist
participants_4_stats_firstInhibitorKill
participants_4_stats_firstInhibitorAssist
participants_4_stats_combatPlayerScore
participants_4_stats_objectivePlayerScore
participants_4_stats_totalPlayerScore
participants_4_stats_totalScoreRank
participants_4_stats_playerScore0
participants_4_stats_playerScore1
participants_4_stats_playerScore2
participants_4_stats_playerScore3
participants_4_stats_playerScore4
participants_4_stats_playerScore5
participants_4_stats_playerScore6
participants_4_stats_playerScore7
participants_4_stats_playerScore8
participants_4_stats_playerScore9
participants_4_stats_perk0
participants_4_stats_perk0Var1
participants_4_stats_perk0Var2
participants_4_stats_perk0Var3
participants_4_stats_perk1
participants_4_stats_perk1Var1
participants_4_stats_perk1Var2
participants_4_stats_perk1Var3
participants_4_stats_perk2
participants_4_stats_perk2Var1
participants_4_stats_perk2Var2
participants_4_stats_perk2Var3
participants_4_stats_perk3
participants_4_stats_perk3Var1
participants_4_stats_perk3Var2
participants_4_stats_perk3Var3
participants_4_stats_perk4
participants_4_stats_perk4Var1
participants_4_stats_perk4Var2
participants_4_stats_perk4Var3
participants_4_stats_perk5
participants_4_stats_perk5Var1
participants_4_stats_perk5Var2
participants_4_stats_perk5Var3
participants_4_stats_perkPrimaryStyle
participants_4_stats_perkSubStyle
participants_4_stats_statPerk0
participants_4_stats_statPerk1
participants_4_stats_statPerk2
participants_4_timeline_participantId
participants_4_timeline_creepsPerMinDeltas_0-10
participants_4_timeline_xpPerMinDeltas_0-10
participants_4_timeline_goldPerMinDeltas_0-10
participants_4_timeline_csDiffPerMinDeltas_0-10
participants_4_timeline_xpDiffPerMinDeltas_0-10
participants_4_timeline_damageTakenPerMinDeltas_0-10
participants_4_timeline_damageTakenDiffPerMinDeltas_0-10
participants_4_timeline_role
participants_4_timeline_lane






------------------------------------------------------------------------------------------
gameDuration
teams_teamId
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_riftHeraldKills













gameId
platformId
gameCreation
gameDuration
queueId
mapId
seasonId
gameVersion
gameMode
gameType
teams_0_teamId
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_vilemawKills
teams_0_riftHeraldKills
teams_0_dominionVictoryScore
teams_1_teamId
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_vilemawKills
teams_1_riftHeraldKills
teams_1_dominionVictoryScore
participants_0_participantId
participants_0_teamId
participants_0_championId
participants_0_spell1Id
participants_0_spell2Id
participants_0_stats_participantId
participants_0_stats_win
participants_0_stats_item0
participants_0_stats_item1
participants_0_stats_item2
participants_0_stats_item

In [89]:
pprint(a)

{'gameCreation': 1611228224946,
 'gameDuration': 1927,
 'gameId': 4938438709,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '11.2.353.8505',
 'mapId': 11,
 'participantIdentities_0_participantId': 1,
 'participantIdentities_0_player_accountId': 'WstIH4CJ4my6D1vfns0X0wdFBjLhtHeHaIuaziEszpRct0OestbX_0mY',
 'participantIdentities_0_player_currentAccountId': 'WstIH4CJ4my6D1vfns0X0wdFBjLhtHeHaIuaziEszpRct0OestbX_0mY',
 'participantIdentities_0_player_currentPlatformId': 'KR',
 'participantIdentities_0_player_matchHistoryUri': '/v1/stats/player_history/KR/2002081850125888',
 'participantIdentities_0_player_platformId': 'KR',
 'participantIdentities_0_player_profileIcon': 4568,
 'participantIdentities_0_player_summonerId': 'BgdbCThuvxo_koDJ7f16wdVBdYwcnDf8PEhxuNa9kv0GrZI',
 'participantIdentities_0_player_summonerName': 'Siod0me',
 'participantIdentities_1_participantId': 2,
 'participantIdentities_1_player_accountId': 'scO_rjJ4EfhJz-Mrt6hj8v30ti4i5IUvnT6EGXyHDqGTnNRRTt

In [ ]:
new branch